<a href="https://colab.research.google.com/github/ReinholdssonJ/squash/blob/main/Squash.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Set up

In [118]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()
gc = gspread.authorize(creds)

from gspread_dataframe import set_with_dataframe

# Convert to a DataFrame and render.
import pandas as pd

In [114]:
squash_sheet = gc.open('squash_records')

rows = squash_sheet.worksheet('Data').get_all_values()
colnames = rows[0]
data = rows[1:]

scores_df = pd.DataFrame(data,
                  columns=colnames)

scores_df.head()
#scores_df.dtypes

,Date,Set,Player,Score,Won_set,Won_game
0,29-04-2024,1,J,11,1,0
1,29-04-2024,1,M,8,0,0
2,29-04-2024,2,J,5,0,0
3,29-04-2024,2,M,11,1,0
4,29-04-2024,3,J,11,1,0


In [55]:
# Clean data

# Remove rows with no scores
scores_df = scores_df[(pd.notnull(scores_df['Score'])) & (scores_df['Score']!="")]

# Only select relevant columns
scores_df = scores_df[['Date', 'Set', 'Player', 'Score']]

# Fix column formats
scores_df['Set'] = scores_df['Set'].astype('int')
scores_df['Score'] = scores_df['Score'].astype('int')
scores_df['Date'] = pd.to_datetime(scores_df['Date']).dt.strftime('%d-%m-%Y')
scores_df.dtypes

Date      object
Set        int64
Player    object
Score      int64
dtype: object

In [47]:
# Define useful objects

j_alias = ['jakob', 'j']
m_alias = ['matteo', 'm']

dates = scores_df['Date'].unique()

In [56]:
scores_df.head(-10)

,Date,Set,Player,Score
0,29-04-2024,1,J,11
1,29-04-2024,1,M,8
2,29-04-2024,2,J,5
3,29-04-2024,2,M,11
4,29-04-2024,3,J,11
...,...,...,...,...
87,20-05-2024,7,M,11
88,24-05-2024,1,J,14
89,24-05-2024,1,M,12
90,24-05-2024,2,J,12


# Functions

In [66]:
def get_set_stats(df, date, set_num):
  # Get data
  date_set = df[(df['Date']==date) & (df['Set']==set_num)]

  # Get the score for the winner and loser
  higher_score = date_set['Score'].max()
  lower_score = date_set['Score'].min()

  # Get the name of the winner
  winner = date_set[date_set['Score'] == higher_score]['Player'].values[0]
  loser = date_set[date_set['Score'] == lower_score]['Player'].values[0]

  # Get the score margin
  score_margin = higher_score - lower_score

  # Dictionary with stats to be returned
  set_stats = {'date': date,
               'set_num': set_num,
               'winner': winner,
               'winner_score': higher_score,
               'loser': loser,
               'loser_score': lower_score,
               'winning_margin': score_margin}

  return(set_stats)

example = get_set_stats(scores_df, '29-04-2024', 2)

example

{'date': '29-04-2024',
 'set_num': 2,
 'winner': 'M',
 'winner_score': 11,
 'loser': 'J',
 'loser_score': 5,
 'winning_margin': 6}

In [99]:
def get_game_stats(df, date):
  # Counters to keep track of the tally over a game.
  game_stats = {'date': date,
                'Jakob': 0,
                'Matteo': 0}

  # Gets the number of sets to loop through
  sets = df[df['Date'] == date]['Set'].unique()

  # Loops through the sets and adds 1 to the tally of Jakob wins or Matteo wins
  for set_num in sets:
    # Get the winner of the set
    set_stats = get_set_stats(df, date, set_num)

    if set_stats['winner'].lower() in j_alias:
      game_stats['Jakob'] += 1

    elif set_stats['winner'].lower() in m_alias:
      game_stats['Matteo'] += 1

  jakob_margin = game_stats['Jakob'] - game_stats['Matteo']

  if jakob_margin > 0:
    game_stats['winner'] = 'Jakob'
  elif jakob_margin == 0:
    game_stats['winner'] = 'Draw'
  else:
    game_stats['winner'] = 'Matteo'

  game_stats['jakob_margin'] = jakob_margin

  return(game_stats)

get_game_stats(scores_df, '29-04-2024')

{'date': '29-04-2024',
 'Jakob': 6,
 'Matteo': 1,
 'winner': 'Jakob',
 'jakob_margin': 5}

# Save game results to spreadsheet

In [101]:
# Create empty data frame to save the stats for each set
set_columns = get_set_stats(scores_df, dates[1], 1).keys()
set_df = pd.DataFrame(columns = set_columns)

# Create empty data frame to save the stats for each game
game_columns = get_game_stats(scores_df, dates[1]).keys()
game_df = pd.DataFrame(columns = game_columns)

# Loop through each date to get stats for each day
for date in dates:
  # Get the stats for the game that day, and append to DataFrame with game stats
  game_stats = pd.DataFrame([get_game_stats(scores_df, date)])
  game_df = pd.concat([game_df, game_stats], ignore_index=True)

  # Loop through each set and get the stats
  num_sets = scores_df[scores_df['Date']==date]['Set'].unique()
  for set_num in num_sets:
    # Get the stats for the set, and append to DataFrame with set stats
    set_stats = pd.DataFrame([get_set_stats(scores_df, date, set_num)])
    set_df = pd.concat([set_df, set_stats], ignore_index=True)

,date,Jakob,Matteo,winner,jakob_margin
0,29-04-2024,6,1,Jakob,5
1,01-05-2024,4,3,Jakob,1
2,10-05-2024,4,0,Jakob,4
3,13-05-2024,1,5,Matteo,-4
4,15-05-2024,3,3,Draw,0
5,17-05-2024,3,4,Matteo,-1
6,20-05-2024,3,4,Matteo,-1
7,24-05-2024,5,2,Jakob,3


In [121]:
set_sheet = squash_sheet.worksheet('Sets')
set_sheet.clear
set_with_dataframe(set_sheet, set_df)

game_sheet = squash_sheet.worksheet('Games')
game_sheet.clear

set_with_dataframe(game_sheet, game_df)